# Ingest ADS Notebook Files (Interactive)

![files_to_database](.\files_to_database.png)

This Notebook will load selected files (from a local or networked file system) into a SQL Server database.

This notebook is designed run _interactively_ and does not require any supporting objects in eltSnap.

The associated notebook: [Ingest ADS Notebook Files (Project)](.\ingest_ads_notebook_files_project.ipynb) can be used to Deploy this notebook capability as a runnable _Project_ into the eltSnap database.

> For best results, verify/update the **Notebook Configuration** code cell below, and then use the '**Run all**' option for this Notebook.
> 
> If the SQL Server commands for PowerShell need to be installled, use: Install-Module -Name SqlServer -AllowClobber
> 
> <span style="background-color: rgb(203, 213, 225); color: rgb(15, 23, 42); font-family: ui-monospace, SFMono-Regular, Menlo, Monaco, Consolas, &quot;Liberation Mono&quot;, &quot;Courier New&quot;, monospace;">To determine PowerShell version, use: (Get-Host).Version</span>

## Notebook Configuration

> If using **SQL Server authentication** provide $username and $pwd in the code cell below:

In [ ]:
# Configuration Parameters
$FolderLocation = 'C:\repos\'   # Update to your Notebook Repository
$ServerInstance = 'localhost'   # Update to your SQL Server Name hosting the 'eltsnap_v2' database
$Database = 'eltsnap_v2'

# Set Begin Time
$BegTime = $(Get-Date)
'Beg Time: ' +  $BegTime

# Show Environment
'Run Values:'
'Folder Location: ' + $FolderLocation
'Server: ' + $ServerInstance
'Database: ' +$Database

# Use SQL Server username and password
$username = "*****"
$pwd = ConvertTo-SecureString -String "*****" -AsPlainText -Force
$cred = New-Object -TypeName System.Management.Automation.PSCredential -ArgumentList $username, $pwd

## Reset staging table

> If using **SQL Server authentication** uncomment the following (6) code cells below to include the parameter: "-Credential $cred" 

In [ ]:
Invoke-Sqlcmd -ServerInstance $ServerInstance -Database $Database -Query "TRUNCATE TABLE [stg].[files]" <# -Credential $cred #>

## Load Notebook Files into table: [stg].[files]


In [ ]:
Get-ChildItem -Path $FolderLocation -Recurse -Include *.ipynb | Select-Object @{l="FullName";e={($_.FullName).Replace($FolderLocation,"")}}, BaseName, @{l="Extension";e={($_.Extension).Replace(".","")}}, LastWriteTime, Length, @{l="Contents";e={(Get-Content $_ -Raw).ToString()}} | Write-SqlTableData -ServerInstance $ServerInstance -DatabaseName $database -SchemaName "stg" -TableName "files" <# -Credential $cred #>

## Load .SQL Files into table: [stg].[files]

In [ ]:
Get-ChildItem -Path $FolderLocation -Recurse -Include *.sql | Select-Object @{l="FullName";e={($_.FullName).Replace($FolderLocation,"")}}, BaseName, @{l="Extension";e={($_.Extension).Replace(".","")}}, LastWriteTime, Length, @{l="Contents";e={(Get-Content $_ -Raw).ToString()}} | Write-SqlTableData -ServerInstance $ServerInstance -DatabaseName $database -SchemaName "stg" -TableName "files" <# -Credential $cred #>

## Merge table: [stg].[files] into: [ads].[files]

In [ ]:
Invoke-Sqlcmd -ServerInstance $ServerInstance -Database $Database -Query "EXEC [ads].[Merge Staged Files];" <# -Credential $cred #>

## Ingest all Notebooks Files into table: [ads].[notebooks]

In [ ]:
Invoke-Sqlcmd -ServerInstance $ServerInstance -Database $Database -Query "EXEC [ads].[Ingest All Notebooks];" <# -Credential $cred #>

## Ingest all SQL Files into table: [ads].[sql_files]

In [ ]:
Invoke-Sqlcmd -ServerInstance $ServerInstance -Database $Database -Query "EXEC [ads].[Ingest All SQL Files];" <# -Credential $cred #>

## Run Summary

In [ ]:
'Beg Time: ' +  $BegTime

$EndTime = $(Get-Date)
'End Time: ' +  $EndTime

$EndTime - $BegTime
